In [3]:
import pandas as pd
import numpy as np
import pandas as pd
from transformers import BertTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [38]:
# Load the dataset
df1 = pd.read_csv(r'/Text/train.csv')

df2 = pd.read_csv(r'/Text/test.csv')

In [ ]:
df2.head()

,Test_ID,Test_Title_name,Test_Title_Description
0,A1,Fears for T N pension after talks,Unions representing workers at Turner Newall...
1,A2,The Race is On: Second Private Team Sets Launc...,"SPACE.com - TORONTO, Canada -- A second\team o..."
2,A3,Ky. Company Wins Grant to Study Peptides (AP),AP - A company founded by a chemistry research...
3,A4,Prediction Unit Helps Forecast Wildfires (AP),AP - It's barely dawn when Mike Fitzpatrick st...
4,A5,Calif. Aims to Limit Farm-Related Smog (AP),AP - Southern California's smog-fighting agenc...


In [ ]:
# BERT Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize(text):
    return tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text, add_special_tokens=True)))

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [5]:
#Combine the text columns to populate Combined_text
df1['Combined_Text'] = df1['Train_Title'] + ' ' + df1['Train_Title_Description']
df2['Combined_Text'] = df2['Test_Title_name'] + ' ' + df2['Test_Title_Description']

In [ ]:
#Tokenization
df1['Tokens'] = df1['Combined_Text'].apply(tokenize)
df2['Tokens'] = df2['Combined_Text'].apply(tokenize)

In [ ]:
# Shared Vocabulary
all_tokens = list(set(df1['Tokens'].sum() + df2['Tokens'].sum()))

In [ ]:
# TF-IDF Vectors
tfidf_vectorizer = TfidfVectorizer(vocabulary=all_tokens, analyzer=lambda x: x)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(df1['Tokens'])
tfidf_matrix_test = tfidf_vectorizer.transform(df2['Tokens'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [ ]:
# Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix_train, tfidf_matrix_test)

In [ ]:
# Store Similarity Scores in a DataFrame
similarity_scores = []
for i, row1 in df1.iterrows():
    for j, row2 in df2.iterrows():
        similarity_scores.append({
            'Train_ID': row1['Train_ID'],
            'Test_ID': row2['Test_ID'],
            'Similarity_Score': cosine_sim[i, j]
        })

In [ ]:
result_df = pd.DataFrame(similarity_scores)

# Print the Result DataFrame
print(result_df)

         Train_ID Test_ID  Similarity_Score
0               1      A1          0.021734
1               1      A2          0.053859
2               1      A3          0.074105
3               1      A4          0.031142
4               1      A5          0.044359
...           ...     ...               ...
7599995      1000   A7596          0.040988
7599996      1000   A7597          0.078123
7599997      1000   A7598          0.020070
7599998      1000   A7599          0.027379
7599999      1000   A7600          0.047308

[7600000 rows x 3 columns]


In [ ]:
# Find the top 20 rows with the highest similarity scores
top_20_rows = result_df.nlargest(20, 'Similarity_Score')

# Print the top 20 rows with the highest similarity scores
print("Top 20 Rows with the Highest Similarity Scores:")
print(top_20_rows)

Top 20 Rows with the Highest Similarity Scores:
         Train_ID Test_ID  Similarity_Score
4765240       628     A41          0.988816
7478467       985     A68          0.970569
7419948       977   A2349          0.953731
7569659       997     A60          0.929856
6923657       912     A58          0.901045
5274447       695     A48          0.898791
3291680       434    A881          0.890525
6300470       830     A71          0.887116
6938861       914     A62          0.884065
661221         88     A22          0.882065
7371386       970   A6987          0.881660
1438662       190   A2263          0.873832
7364701       970    A302          0.872667
1431062       189   A2263          0.871111
7364826       970    A427          0.865577
6840061       901     A62          0.861352
7366358       970   A1959          0.860142
7369803       970   A5404          0.860011
1600721       211   A4722          0.851056
7371190       970   A6791          0.845025


In [ ]:
#Roberta


In [7]:
from transformers import RobertaTokenizer, RobertaModel
import torch

In [8]:
#Roberta Tokenization
roberta_tokenization = RobertaTokenizer.from_pretrained('roberta-base')
def rob_tok(text):
  return roberta_tokenization.tokenize(roberta_tokenization.decode(roberta_tokenization.encode(text, add_special_tokens=True)))

In [9]:
#Combine the text columns to populate Combined_text
df1['Combined_Text'] = df1['Train_Title'] + ' ' + df1['Train_Title_Description']
df2['Combined_Text'] = df2['Test_Title_name'] + ' ' + df2['Test_Title_Description']

In [16]:
#Tokenization
df1['Tokens'] = df1['Combined_Text'].apply(rob_tok)
df2['Tokens'] = df2['Combined_Text'].apply(rob_tok)

In [17]:
# Shared Vocabulary
all_tokens = list(set(df1['Tokens'].sum() + df2['Tokens'].sum()))


In [18]:
# TF-IDF Vectors
tfidf_vectorizer = TfidfVectorizer(vocabulary=all_tokens, analyzer=lambda x: x)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(df1['Tokens'])
tfidf_matrix_test = tfidf_vectorizer.transform(df2['Tokens'])

In [13]:
# Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix_train, tfidf_matrix_test)

In [19]:
# Store Similarity Scores in a DataFrame
similarity_scores = []
for i, row1 in df1.iterrows():
    for j, row2 in df2.iterrows():
        similarity_scores.append({
            'Train_ID': row1['Train_ID'],
            'Test_ID': row2['Test_ID'],
            'Similarity_Score': cosine_sim[i, j]
        })

In [20]:
result_df = pd.DataFrame(similarity_scores)

# Print the Result DataFrame
print(result_df)

         Train_ID Test_ID  Similarity_Score
0               1      A1          0.013410
1               1      A2          0.049334
2               1      A3          0.039664
3               1      A4          0.026165
4               1      A5          0.040676
...           ...     ...               ...
7599995      1000   A7596          0.022038
7599996      1000   A7597          0.060368
7599997      1000   A7598          0.009059
7599998      1000   A7599          0.024354
7599999      1000   A7600          0.037847

[7600000 rows x 3 columns]


In [21]:
# Find the top 20 rows with the highest similarity scores
top_20_rows = result_df.nlargest(20, 'Similarity_Score')

# Print the top 20 rows with the highest similarity scores
print("Top 20 Rows with the Highest Similarity Scores Roberta:")
print(top_20_rows)

Top 20 Rows with the Highest Similarity Scores Roberta:
         Train_ID Test_ID  Similarity_Score
7478467       985     A68          0.965297
7419948       977   A2349          0.939176
4765240       628     A41          0.929424
7569659       997     A60          0.902969
661221         88     A22          0.893902
6300470       830     A71          0.885350
6923657       912     A58          0.879008
7371386       970   A6987          0.835476
6840061       901     A62          0.827674
3291680       434    A881          0.823658
1438662       190   A2263          0.821236
7364701       970    A302          0.820955
7364826       970    A427          0.818570
6938861       914     A62          0.817613
1431062       189   A2263          0.814616
1598555       211   A2556          0.808270
3427631       452     A32          0.806846
1601896       211   A5897          0.806234
2799386       369   A2587          0.804330
7366358       970   A1959          0.803450


In [29]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.9 MB/s eta 0:00:00


In [22]:
from transformers import XLNetTokenizer, XLNetModel


In [23]:
#XLNet Tokenization
XL_net_tokenization = XLNetTokenizer.from_pretrained('xlnet-base-cased')
def xl_tok(text):
  return XL_net_tokenization.tokenize(XL_net_tokenization.decode(XL_net_tokenization.encode(text, add_special_tokens = True)))

In [24]:
#Combine the text columns to populate Combined_text
df1['Combined_Text'] = df1['Train_Title'] + ' ' + df1['Train_Title_Description']
df2['Combined_Text'] = df2['Test_Title_name'] + ' ' + df2['Test_Title_Description']

In [27]:
#Tokenization
df1['Tokens'] = df1['Combined_Text'].apply(xl_tok)
df2['Tokens'] = df2['Combined_Text'].apply(xl_tok)

In [28]:
# Shared Vocabulary
all_tokens = list(set(df1['Tokens'].sum() + df2['Tokens'].sum()))


In [29]:
# TF-IDF Vectors
tfidf_vectorizer = TfidfVectorizer(vocabulary=all_tokens, analyzer=lambda x: x)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(df1['Tokens'])
tfidf_matrix_test = tfidf_vectorizer.transform(df2['Tokens'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [30]:
# Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix_train, tfidf_matrix_test)

In [31]:
# Store Similarity Scores in a DataFrame
similarity_scores = []
for i, row1 in df1.iterrows():
    for j, row2 in df2.iterrows():
        similarity_scores.append({
            'Train_ID': row1['Train_ID'],
            'Test_ID': row2['Test_ID'],
            'Similarity_Score': cosine_sim[i, j]
        })

In [32]:
result_df = pd.DataFrame(similarity_scores)

# Print the Result DataFrame
print(result_df)

         Train_ID Test_ID  Similarity_Score
0               1      A1          0.039207
1               1      A2          0.068992
2               1      A3          0.067503
3               1      A4          0.047020
4               1      A5          0.072738
...           ...     ...               ...
7599995      1000   A7596          0.028881
7599996      1000   A7597          0.086329
7599997      1000   A7598          0.013387
7599998      1000   A7599          0.054415
7599999      1000   A7600          0.051160

[7600000 rows x 3 columns]


In [33]:
# Find the top 20 rows with the highest similarity scores
top_20_rows = result_df.nlargest(20, 'Similarity_Score')

# Print the top 20 rows with the highest similarity scores
print("Top 20 Rows with the Highest Similarity Scores xlnet:")
print(top_20_rows)

Top 20 Rows with the Highest Similarity Scores xlnet:
         Train_ID Test_ID  Similarity_Score
7478467       985     A68          0.966486
7419948       977   A2349          0.941347
4765240       628     A41          0.911415
7569659       997     A60          0.909456
6923657       912     A58          0.899315
661221         88     A22          0.892913
6300470       830     A71          0.888641
7371386       970   A6987          0.860610
7364826       970    A427          0.855084
7364701       970    A302          0.846008
3291680       434    A881          0.844265
7366358       970   A1959          0.839695
1601896       211   A5897          0.837494
2799386       369   A2587          0.836454
1598555       211   A2556          0.834818
1600721       211   A4722          0.834340
6840061       901     A62          0.828756
1639896       216   A5897          0.825184
2801981       369   A5182          0.824258
6938861       914     A62          0.822062


In [34]:
from transformers import GPT2Tokenizer, GPT2Model

In [36]:
#GPT2 Tokenization
GPT2_tokenization = GPT2Tokenizer.from_pretrained('gpt2')
def GPT2_tok(text):
  return GPT2_tokenization.tokenize(GPT2_tokenization.decode(GPT2_tokenization.encode(text, add_special_tokens = True)))

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [39]:
#Combine the text columns to populate Combined_text
df1['Combined_Text'] = df1['Train_Title'] + ' ' + df1['Train_Title_Description']
df2['Combined_Text'] = df2['Test_Title_name'] + ' ' + df2['Test_Title_Description']

In [40]:
#Tokenization
df1['Tokens'] = df1['Combined_Text'].apply(GPT2_tok)
df2['Tokens'] = df2['Combined_Text'].apply(GPT2_tok)

In [41]:
# Shared Vocabulary
all_tokens = list(set(df1['Tokens'].sum() + df2['Tokens'].sum()))


In [42]:
# TF-IDF Vectors
tfidf_vectorizer = TfidfVectorizer(vocabulary=all_tokens, analyzer=lambda x: x)
tfidf_matrix_train = tfidf_vectorizer.fit_transform(df1['Tokens'])
tfidf_matrix_test = tfidf_vectorizer.transform(df2['Tokens'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [43]:
# Cosine Similarity
cosine_sim = cosine_similarity(tfidf_matrix_train, tfidf_matrix_test)

In [44]:
# Store Similarity Scores in a DataFrame
similarity_scores = []
for i, row1 in df1.iterrows():
    for j, row2 in df2.iterrows():
        similarity_scores.append({
            'Train_ID': row1['Train_ID'],
            'Test_ID': row2['Test_ID'],
            'Similarity_Score': cosine_sim[i, j]
        })

In [45]:
result_df = pd.DataFrame(similarity_scores)

# Print the Result DataFrame
print(result_df)

         Train_ID Test_ID  Similarity_Score
0               1      A1          0.011472
1               1      A2          0.047990
2               1      A3          0.038022
3               1      A4          0.024807
4               1      A5          0.039060
...           ...     ...               ...
7599995      1000   A7596          0.020674
7599996      1000   A7597          0.059441
7599997      1000   A7598          0.007678
7599998      1000   A7599          0.023483
7599999      1000   A7600          0.036408

[7600000 rows x 3 columns]


In [47]:
# Find the top 20 rows with the highest similarity scores
top_20_rows = result_df.nlargest(20, 'Similarity_Score')

# Print the top 20 rows with the highest similarity scores
print("Top 20 Rows with the Highest Similarity Scores gpt2:")
print(top_20_rows)

Top 20 Rows with the Highest Similarity Scores gpt2:
         Train_ID Test_ID  Similarity_Score
7478467       985     A68          0.965251
7419948       977   A2349          0.939119
4765240       628     A41          0.929331
7569659       997     A60          0.902822
661221         88     A22          0.893640
6300470       830     A71          0.885217
6923657       912     A58          0.878880
7371386       970   A6987          0.835461
6840061       901     A62          0.827428
3291680       434    A881          0.823497
1438662       190   A2263          0.821119
7364701       970    A302          0.820919
7364826       970    A427          0.818533
6938861       914     A62          0.817383
1431062       189   A2263          0.814488
1598555       211   A2556          0.808208
3427631       452     A32          0.806606
1601896       211   A5897          0.806171
2799386       369   A2587          0.804318
7366358       970   A1959          0.803411
